In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob

from sklearn.model_selection import train_test_split
from sklearn import metrics

import cv2
import gc
import os

import tensorflow as tf
from tensorflow import keras
from keras import layers



In [3]:
path = 'lung_colon_image_set/lung_image_sets'
classes = os.listdir(path)
classes


['lung_aca', 'lung_n', 'lung_scc']

In [ ]:
path = 'lung_colon_image_set/lung_image_sets'

for cat in classes:
	image_dir = f'{path}/{cat}'
	images = os.listdir(image_dir)

	fig, ax = plt.subplots(1, 3, figsize=(15, 5))
	fig.suptitle(f'Images for {cat} category . . . .', fontsize=20)

	for i in range(3):
		k = np.random.randint(0, len(images))
		img = np.array(Image.open(f'{path}/{cat}/{images[k]}'))
		ax[i].imshow(img)
		ax[i].axis('off')
	plt.show()


In [18]:
IMG_SIZE =128
SPLIT = 0.2
EPOCHS = 10
BATCH_SIZE = 64


In [23]:
X = []
Y = []

for i, cat in enumerate(classes):
	images = glob(f'{path}/{cat}/*.jpeg')

	for image in images:
		img = cv2.imread(image)
		
		X.append(cv2.resize(img, (IMG_SIZE, IMG_SIZE)))
		Y.append(i)

X = np.asarray(X)
one_hot_encoded_Y = pd.get_dummies(Y).values


In [24]:
X_train, X_val, Y_train, Y_val = train_test_split(X, one_hot_encoded_Y, test_size = SPLIT,random_state = 2022)
print(X_train.shape, X_val.shape)


(12000, 128, 128, 3) (3000, 128, 128, 3)


In [9]:
model = keras.models.Sequential([
	layers.Conv2D(filters=32,
				kernel_size=(5, 5),
				activation='relu',
				input_shape=(IMG_SIZE,
							IMG_SIZE,
							3),
				padding='same'),
	layers.MaxPooling2D(2, 2),

	layers.Conv2D(filters=64,
				kernel_size=(3, 3),
				activation='relu',
				padding='same'),
	layers.MaxPooling2D(2, 2),

	layers.Conv2D(filters=128,
				kernel_size=(3, 3),
				activation='relu',
				padding='same'),
	layers.MaxPooling2D(2, 2),

	layers.Flatten(),
	layers.Dense(256, activation='relu'),
	layers.BatchNormalization(),
	layers.Dense(128, activation='relu'),
	layers.Dropout(0.3),
	layers.BatchNormalization(),
	layers.Dense(3, activation='softmax')
])


c:\Users\Shashank kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [ ]:
model.summary()


In [ ]:
keras.utils.plot_model(
	model,
	show_shapes = True,
	show_dtype = True,
	show_layer_activations = True
)


In [12]:
model.compile(
	optimizer = 'adam',
	loss = 'categorical_crossentropy',
	metrics = ['accuracy']
)


In [13]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


class myCallback(tf.keras.callbacks.Callback):
	def on_epoch_end(self, epoch, logs={}):
		if logs.get('val_accuracy') > 0.90:
			print('\n Validation accuracy has reached upto \
					90% so, stopping further training.')
			self.model.stop_training = True


es = EarlyStopping(patience=3,
				monitor='val_accuracy',
				restore_best_weights=True)

lr = ReduceLROnPlateau(monitor='val_loss',
					patience=2,
					factor=0.5,
					verbose=1)


In [ ]:
history = model.fit(X_train, Y_train,
					validation_data = (X_val, Y_val),
					batch_size = BATCH_SIZE,
					epochs = EPOCHS,
					verbose = 1,
					callbacks = [es, lr, myCallback()])


In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:,['loss','val_loss']].plot()
history_df.loc[:,['accuracy','val_accuracy']].plot()
plt.show()


In [ ]:
Y_pred = model.predict(X_val)
Y_val = np.argmax(Y_val, axis=1)
Y_pred = np.argmax(Y_pred, axis=1)


In [ ]:
metrics.confusion_matrix(Y_val, Y_pred)


In [ ]:
print(metrics.classification_report(Y_val, Y_pred,
									target_names=classes))


In [ ]:
model.save("lung_cancer_model.h5")
model.save("lung_cancer_model.keras")

In [22]:
import os
import cv2
import numpy as np
import tensorflow as tf

# ─── 1) Load your model ───────────────────────────────────────────────────────
# No need to compile since we're only doing inference
model = tf.keras.models.load_model('lung_cancer_model.h5', compile=False)

# ─── 2) Helper: preprocess to match training (BGR [0–255], no scaling) ─────
def preprocess_image_bgr_noscale(path, img_size=128):
    """
    Reads an image as BGR uint8 [0–255], resizes to (img_size, img_size),
    and adds a batch dimension—NO RGB conversion, NO float32 scaling.
    Returns a numpy array of shape (1, img_size, img_size, 3).
    """
    img = cv2.imread(path)  # BGR, uint8 [0–255]
    if img is None:
        raise FileNotFoundError(f"Could not read image at {path!r}")
    img = cv2.resize(img, (img_size, img_size))
    # keep dtype uint8; the model was trained on these raw values
    return np.expand_dims(img, axis=0)

# ─── 3) Inference function ──────────────────────────────────────────────────
def predict_image(path, img_size=128, class_names=None):
    """
    Preprocesses + predicts a single image.
    Prints softmax scores and class index (and name if provided).
    """
    x = preprocess_image_bgr_noscale(path, img_size=img_size)
    preds = model.predict(x)
    cls_idx = np.argmax(preds, axis=1)[0]
    scores = preds.flatten()
    print(f"\n[{os.path.basename(path)}]")
    print("  Softmax scores:", np.round(scores, 4))
    if class_names:
        print("  → Predicted:", cls_idx, class_names[cls_idx])
    else:
        print("  → Predicted class index:", cls_idx)
    return cls_idx, scores

# ─── 4) Define your class-label mapping ──────────────────────────────────────
# (must match `classes = os.listdir(path)` ordering from training)
class_names = ['lung_aca', 'lung_n', 'lung_scc']  # example order

# ─── 5) Test on one or more images ───────────────────────────────────────────
test_paths = [
    'lung_colon_image_set\lung_image_sets\lung_scc\lungscc1.jpeg',
    'lung_colon_image_set/lung_image_sets/lung_n/lungn1.jpeg',
    # add more paths here...
]

for tp in test_paths:
    predict_image(tp, img_size=128, class_names=class_names)

# ─── 6) (Optional) Sanity-check on random noise ─────────────────────────────
noise = (np.random.randint(0, 256, size=(1,128,128,3), dtype=np.uint8))
preds_noise = model.predict(noise)
print("\n[Random noise]")
print("  Softmax scores:", np.round(preds_noise.flatten(),4))
print("  → Class index:", np.argmax(preds_noise, axis=1)[0])


<>:48: SyntaxWarning: invalid escape sequence '\l'
<>:48: SyntaxWarning: invalid escape sequence '\l'
C:\Users\Shashank kumar\AppData\Local\Temp\ipykernel_19388\1130306805.py:48: SyntaxWarning: invalid escape sequence '\l'
  'lung_colon_image_set\lung_image_sets\lung_scc\lungscc1.jpeg',


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step

[lungscc1.jpeg]
  Softmax scores: [0. 0. 1.]
  → Predicted: 2 lung_scc
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step

[lungn1.jpeg]
  Softmax scores: [0.0027 0.9973 0.    ]
  → Predicted: 1 lung_n
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

[Random noise]
  Softmax scores: [0. 1. 0.]
  → Class index: 1
